<a href="https://colab.research.google.com/github/raghavnani/2018fall/blob/master/earnings_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def get_news(publishedAt , queryString = None):
  import urllib.request
  import json

  API = "https://api.newsfilter.io/public/actions?token=irhkg02jqfbajhelbnp99723sti4ioroisdfoauoweiuoeaei66rsyolszgdzxzv"


  if queryString:
    payload = {
      'type': "filterArticles",
      'isPublic': 'true',
      'queryString': queryString + ' AND ' + publishedAt,
      'from': 0,
      'size': 10000,
      'order' : 'desc'
    }
  else:
    payload = {
      'type': "filterArticles",
      'queryString': publishedAt,
      'isPublic': 'true',
      'from': 0,
      'size': 10000,
      'publishedAt': publishedAt,
      'order' : 'desc'

    }

  # Format your payload to JSON bytes
  jsondata = json.dumps(payload)
  jsondataasbytes = jsondata.encode('utf-8')  # needs to be bytes

  req = urllib.request.Request(API)

  req.add_header('Content-Type', 'application/json; charset=utf-8')
  # Set the correct length of your request
  req.add_header('Content-Length', len(jsondataasbytes))

  response = urllib.request.urlopen(req, jsondataasbytes)

  res_body = response.read()

  articles = json.loads(res_body.decode("utf-8"))

  return articles



In [2]:
!pip install tensorflow-gpu==2.1.0

     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 3.8MB 50.9MB/s 
     |████████████████████████████████| 450kB 77.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
import tensorflow as tf

In [4]:
tf.__version__

'2.1.0'

In [0]:
import pandas as pd

In [0]:
earnings  = get_news("(title:earnings OR description:earnings) OR (title:quarter)", 'publishedAt:[2019-03-01 TO 2019-09-30]')

In [0]:
earnings_df = pd.DataFrame(earnings['articles'])

In [8]:
earnings_df

,source,categories,symbols,markets,title,content,description,url,publishedAt,id,urlToImage,author,important,country,time,watchCount,publishedAtNorm,watchCountNorm,ranking,tweetedAt,chartUrl
0,"{'id': 'reuters', 'name': 'Reuters'}",[],[JPM],[],Third quarter could mark turning point in U.S....,,As investors prepare for U.S. corporations to ...,https://newsfilter.io/articles/third-quarter-c...,2019-09-30T21:39:22+00:00,6c922d33c3b131be8fed0c3dc475fa50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': 'seekingAlpha', 'name': 'Seeking Alpha'}",[],[SFIX],[],Stitch Fix Q4 Earnings Preview,,Stitch Fix (NASDAQ:SFIX) is scheduled to annou...,https://newsfilter.io/articles/stitch-fix-q4-e...,2019-09-30T17:35:00-04:00,9a0504e56c19b942363bed618cff67a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': 'seekingAlpha', 'name': 'Seeking Alpha'}",[],[UNFI],[],United Natural Foods Q4 2019 Earnings Preview,,United Natural Foods (NYSE:UNFI) is scheduled ...,https://newsfilter.io/articles/united-natural-...,2019-09-30T17:35:00-04:00,cea449773473c9122d9ad2ffa200f2d0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': 'businesswire', 'name': 'BusinessWire'}",[],"[PCH, PCH]",[],PotlatchDeltic Scheduled to Release Third Quar...,,"SPOKANE, Wash.--(BUSINESS WIRE)--PotlatchDelti...",https://newsfilter.io/articles/potlatchdeltic-...,2019-09-30T21:33:00+00:00,36081a072dd7ed72081afb0810f8ffd6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': 'prNewswire', 'name': 'PR Newswire'}",[],[LEA],[],Lear Announces Date for Third Quarter 2019 Ear...,,Lear Corporation (NYSE: LEA) will hold a confe...,https://newsfilter.io/articles/lear-announces-...,2019-09-30T17:30:00-04:00,672eeedda7a0ecba176000de2841f2c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,"{'id': 'marketwatch', 'name': 'MarketWatch'}",[],[],[],Colgate-Palmolive tops estimates in latest qua...,,Colgate-Palmolive Co. said Friday it had net ...,https://newsfilter.io/articles/colgate-palmoli...,2019-04-26T07:05:01-04:00,369f9b60dd2c0ee884dd6f301fb24aab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9902,"{'id': 'prNewswire', 'name': 'PR Newswire'}",[],[],[],Universal Technical Institute Schedules Fiscal...,,"Universal Technical Institute, Inc. (NYSE: UTI...",https://newsfilter.io/articles/universal-techn...,2019-04-26T07:05:00-04:00,7b6e7838d3e58143ec9846c865e479ef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9903,"{'id': 'marketwatch', 'name': 'MarketWatch'}",[],[],[],"Amazon profit doubles to a record high, Prime ...",,Amazon.com Inc. doubled its profit in the firs...,https://newsfilter.io/articles/amazon-profit-d...,2019-04-26T06:53:40-04:00,873cdf1de563e6312f9fe9fd644708e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9904,"{'id': 'reuters', 'name': 'Reuters'}",[],[],[],AutoNation quarterly profit beats Wall Street ...,"(Reuters) - AutoNation Inc, the largest U.S. a...","AutoNation Inc, the largest U.S. auto dealersh...",https://newsfilter.io/articles/autonation-quar...,2019-04-26T10:37:49Z,9cd76f98a83c2da7dc353051ab3f7b42,https://s4.reutersmedia.net/resources_v2/image...,Reuters Editorial,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
all_news  = get_news("publishedAt:[2019-03-01 TO 2019-09-29]")

In [0]:
all_news_df = pd.DataFrame(all_news['articles'])

In [11]:
all_news_df['publishedAt']

0            2019-09-29T23:51:00Z
1       2019-09-29T19:45:00-04:00
2            2019-09-29T23:44:00Z
3       2019-09-29T23:38:59+00:00
4       2019-09-29T19:36:00-04:00
                  ...            
9790    2019-09-25T15:56:51-04:00
9791    2019-09-25T15:56:37-04:00
9792    2019-09-25T15:56:35-04:00
9793    2019-09-25T15:56:13-04:00
9794    2019-09-25T15:55:54-04:00
Name: publishedAt, Length: 9795, dtype: object

In [12]:
earnings_df['publishedAt']

0       2019-09-30T21:39:22+00:00
1       2019-09-30T17:35:00-04:00
2       2019-09-30T17:35:00-04:00
3       2019-09-30T21:33:00+00:00
4       2019-09-30T17:30:00-04:00
                  ...            
9901    2019-04-26T07:05:01-04:00
9902    2019-04-26T07:05:00-04:00
9903    2019-04-26T06:53:40-04:00
9904         2019-04-26T10:37:49Z
9905    2019-04-26T06:30:00-04:00
Name: publishedAt, Length: 9906, dtype: object

In [13]:
all_news_df.shape, earnings_df.shape

((9795, 15), (9906, 21))

In [0]:
other_news = all_news_df[~all_news_df['id'].isin(earnings_df['id'])]


In [0]:
earnings_df = earnings_df[earnings_df['source'] != {'id': 'sec-api', 'name': 'SEC'}]

In [0]:
other_news =other_news[other_news['source'] != {'id': 'sec-api', 'name': 'SEC'}]

In [0]:
earnings_df = earnings_df.sample(5200)

In [18]:
other_news.shape , earnings_df.shape

((5200, 15), (5200, 21))

In [19]:
other_news.columns

Index(['source', 'categories', 'symbols', 'markets', 'title', 'content',
       'description', 'url', 'urlToImage', 'author', 'important',
       'publishedAt', 'country', 'id', 'details'],
      dtype='object')

In [20]:
earnings_df.columns

Index(['source', 'categories', 'symbols', 'markets', 'title', 'content',
       'description', 'url', 'publishedAt', 'id', 'urlToImage', 'author',
       'important', 'country', 'time', 'watchCount', 'publishedAtNorm',
       'watchCountNorm', 'ranking', 'tweetedAt', 'chartUrl'],
      dtype='object')

In [0]:
earnings_df['output'] = 1

In [0]:
other_news['output'] = 0

In [0]:
data = earnings_df[['title', 'description','output']].append(other_news[['title', 'description','output']]).reset_index(drop = True)

In [0]:
data = data.sample(frac=1).reset_index(drop=True)

In [25]:
data['title'] + '. ' + data['description'][0]

0        Pinterest stock gains sharply ahead of post-IP...
1        Uber Stock Is Slumping After Earnings Because ...
2        Tiffany Is Rising Because Dividend Hikes Have ...
3        Chewy Q2 2020 Earnings Preview. Shares of Pint...
4        Bank Of America Likes Funko After 2019 Guidanc...
                               ...                        
10395    SoFi Says Millennials Can Invest In Their Reti...
10396    7 Stocks To Watch For August 13, 2019. Shares ...
10397    Enbridge Re-affirms Plan to Proceed with Contr...
10398    Earnings Scheduled For September 11, 2019. Sha...
10399    Triple-S Management Corporation Schedules Seco...
Name: title, Length: 10400, dtype: object

In [0]:
text = data['title'][0] + '. ' + data['description'][0]

In [27]:
len(text.split())

49

In [0]:
import numpy as np

In [29]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [30]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [31]:
!pip install transformers

     |████████████████████████████████| 501kB 19.7MB/s 
     |████████████████████████████████| 1.0MB 55.1MB/s 
     |████████████████████████████████| 870kB 51.6MB/s 
     |████████████████████████████████| 3.7MB 63.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=99e4fdfb92ccb5d7903f6a774dfb8c0b6534b5f61df10ea6821c4d83b84b3db9
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
from tqdm import tqdm 

In [0]:
# Mask ids: for every token to mask out tokens used only for the sequence padding (so every sequence has the same length).
# zero padding
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


In [0]:
# Segment ids: 0 for one-sentence sequence, 1 if there are two sentences in the sequence and it is the second one

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [0]:
#Token ids: for every token in the sentence. We restore it from the BERT vocab dictionary

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids


In [0]:
max_seq_length = 64

In [39]:
data.groupby('output').count()

,title,description
output,,
0,5200,5200
1,5200,5157


In [0]:
data['input'] = data['title'] +'. '+ data['description']

In [41]:
data.head()

,title,description,output,input
0,Pinterest stock gains sharply ahead of post-IP...,Shares of Pinterest Inc. rose more than 7% at...,1,Pinterest stock gains sharply ahead of post-IP...
1,Uber Stock Is Slumping After Earnings Because ...,Uber said it lost $2.01 a share on revenue tha...,1,Uber Stock Is Slumping After Earnings Because ...
2,Tiffany Is Rising Because Dividend Hikes Have ...,The luxury retailer beat first-quarter earning...,1,Tiffany Is Rising Because Dividend Hikes Have ...
3,Chewy Q2 2020 Earnings Preview,Chewy (NYSE:CHWY) is scheduled to announce Q2 ...,1,Chewy Q2 2020 Earnings Preview. Chewy (NYSE:CH...
4,Bank Of America Likes Funko After 2019 Guidanc...,Funko Inc (NASDAQ: FNKO) shares are moving hig...,1,Bank Of America Likes Funko After 2019 Guidanc...


In [0]:
data = data.dropna()

In [43]:
data.groupby('output').count()

,title,description,input
output,,,
0,5200,5200,5200
1,5157,5157,5157


In [0]:
max_seq_length = 64  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

In [0]:
from transformers import TFBertModel

In [46]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [47]:
bert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [0]:
output = bert_model([input_word_ids, input_mask, segment_ids])
X = tf.keras.layers.Dense(128, 'relu')(output[1])
Y = tf.keras.layers.Dropout(0.4)(X)
output1 = tf.keras.layers.Dense(1, 'sigmoid')(Y)
model1 = tf.keras.models.Model([input_word_ids, input_mask, segment_ids], output1)

In [0]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 64)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 64, 768), (N 109482240   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
!pip install tfa-nightly

     |████████████████████████████████| 1.0MB 5.1MB/s 


In [0]:
import tensorflow_addons as tfa

In [0]:
optimizer = tfa.optimizers.AdamW(learning_rate = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  epsilon = 1e-8, # args.adam_epsilon  - default is 1e-8.
                  weight_decay = 1e-7
                )

In [0]:
model1.compile(optimizer= optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
labels = data['output'].values

In [0]:
nlp(cleaned_data['Cleaned_input'][0])[6].is_digit

False

In [0]:
import re

In [0]:
re.sub(r'([^a-zA-Z\s]+?)', '', "'226710': 'Kevi#$%^&n Werbach'")

' Kevin Werbach'

In [0]:
import spacy #load spacy
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("english")



def normalize(comment, lowercase = True):
  count = True
  if lowercase:
      comment = comment.lower()

  comment = re.sub(r'([^a-zA-Z\s]+?)', ' ', comment)
  comment = nlp(comment)
  lemmatized = list()
  for word in comment:
      if not (word.is_punct or word.is_digit) :
        lemma = word.lemma_.strip()

        if lemma and count:
          if  ((lemma not in stops)):
              if (re.compile(r"\(|\[|\<").search(lemma) != None):
                if (re.compile(r"\(|\[|\<").search(lemma).span()[0] == 0):
                  count = False;
                else:
                  lemmatized.append(lemma.split('(')[0])
                  count = False
              else:
                lemmatized.append(lemma)

        if (not count) and (re.compile(r"\)|\]|\>").search(lemma) != None):
          count = True
 

  return " ".join(lemmatized).replace('\ufeff1','1')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
data['input'][0]

'Jefferies Financial Group EPS misses by $0.19, misses on revenue. Jefferies Financial Group (NYSE:JEF): Q3 GAAP EPS of $0.15 misses by $0.19.Revenue of $856.78M (-25.5% Y/Y) misses by $136.42M.Press Release'

In [0]:
normalize(data['input'][0])

'jefferies financial group eps miss miss revenue jefferies financial group nyse jef q gaap eps miss revenue miss press release'

In [0]:
def pre_process_text(target_df):

  target_df.dropna(inplace = True)

  target_df = target_df.reset_index()

  target_df['Cleaned_input'] = target_df['input'].apply(normalize, lowercase=True)

  return target_df


In [0]:
from multiprocessing import Pool
def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    
    return df.reset_index(drop = True)

In [0]:
cleaned_data = parallelize_dataframe(data,pre_process_text)

In [0]:
cleaned_data['Cleaned_input'][0]

'jefferies financial group eps miss miss revenue jefferies financial group nyse jef q gaap eps miss revenue miss press release'

In [0]:
length=[]
for i in cleaned_data['Cleaned_input']:
  length.append(len(i.split()))


In [0]:
np.mean(length)

33.1554739588366

In [0]:
cleaned_data['Cleaned_input'][16]

'wwe superstar becky lynch big show feature million post cereal box post consumer brand wwe nyse wwe today announce expand partnership bring thrill excitement wwe cereal box'

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
ids = []
masks=[]
segments=[]

# For every sentence...
for s in tqdm(cleaned_data['Cleaned_input']):
    
    stokens = tokenizer.tokenize(s, lower= True)
    stokens = stokens[0:30]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    ids.append(input_ids)
    masks.append(input_masks)
    segments.append(input_segments)


100%|██████████| 10349/10349 [00:08<00:00, 1190.21it/s]


In [0]:
ids = np.array(ids)
masks = np.array(masks)
segments = np.array(segments)

In [0]:
model1.fit([ids, masks, segments], labels, epochs= 4 ,validation_split=0.2, batch_size= 64 , shuffle= True)

Train on 8279 samples, validate on 2070 samples
Epoch 1/4
8279/8279 [==============================] - 82s 10ms/sample - loss: 0.1935 - accuracy: 0.9256 - val_loss: 0.0903 - val_accuracy: 0.9696
Epoch 2/4
8279/8279 [==============================] - 66s 8ms/sample - loss: 0.0736 - accuracy: 0.9742 - val_loss: 0.0763 - val_accuracy: 0.9725
Epoch 3/4
8279/8279 [==============================] - 66s 8ms/sample - loss: 0.0485 - accuracy: 0.9833 - val_loss: 0.0721 - val_accuracy: 0.9783
Epoch 4/4
8279/8279 [==============================] - 66s 8ms/sample - loss: 0.0299 - accuracy: 0.9907 - val_loss: 0.0925 - val_accuracy: 0.9783


In [0]:
test  = get_news("(title:earnings OR description:earnings) OR (title:quarter)", 'publishedAt:[2019-10-01 TO 2019-12-31]')

In [0]:
other  = get_news('publishedAt:[2019-10-01 TO 2019-12-31]')

In [0]:
test_df = pd.DataFrame(test['articles'])

In [0]:
other_df = pd.DataFrame(other['articles'])

In [0]:
other_df =other_df[other_df['source'] != {'id': 'sec-api', 'name': 'SEC'}]
test_df =test_df[test_df['source'] != {'id': 'sec-api', 'name': 'SEC'}]

In [0]:
other = other_df[~other_df['id'].isin(test_df['id'])]

In [0]:
test_df.shape, other.shape

((9867, 20), (4832, 20))

In [0]:
test_df['output'] =1
other['output'] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
input_df = test_df.append(other).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [0]:
input_df

,author,categories,content,country,description,details,id,imageUrl,important,industries,markets,output,publishedAt,sectors,source,symbols,thumbnail,title,url,usFdaDrugApprovalDetails,usFdaType,videoDuration,videoType,videoUrl
0,NaN,[],,NaN,Analysts forecast that Grand Canyon Education ...,NaN,90c035791e123567a719acfa96ab1865,https://www.americanbankingnews.com/wp-content...,NaN,[Education & Training Services],[],1,2019-12-31T23:35:37+00:00,[Consumer Defensive],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[LOPE],NaN,$213.92 Million in Sales Expected for Grand Ca...,https://newsfilter.io/articles/21392-million-i...,NaN,NaN,NaN,NaN,NaN
1,NaN,[],,NaN,Brokerages forecast that Aramark (NYSE:ARMK) w...,NaN,b980f68994c1b9cef74f8ef97e47ae65,https://www.americanbankingnews.com/wp-content...,NaN,[Restaurants],[],1,2019-12-31T23:16:53+00:00,[Consumer Cyclical],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[ARMK],NaN,$4.31 Billion in Sales Expected for Aramark (N...,https://newsfilter.io/articles/431-billion-in-...,NaN,NaN,NaN,NaN,NaN
2,NaN,[],,NaN,Wall Street brokerages expect that QuinStreet ...,NaN,42bb835b7291279e1baf8016aea39771,https://www.americanbankingnews.com/wp-content...,NaN,[Internet Content & Information],[],1,2019-12-31T23:14:51+00:00,[Technology],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[QNST],NaN,$0.14 Earnings Per Share Expected for QuinStre...,https://newsfilter.io/articles/014-earnings-pe...,NaN,NaN,NaN,NaN,NaN
3,NaN,[],,NaN,Equities analysts expect Kindred Biosciences I...,NaN,38194c600105856008546c8568a1284a,https://www.americanbankingnews.com/wp-content...,NaN,[Biotechnology],[],1,2019-12-31T22:48:50+00:00,[Healthcare],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[KIN],NaN,Zacks: Analysts Expect Kindred Biosciences Inc...,https://newsfilter.io/articles/zacks-analysts-...,NaN,NaN,NaN,NaN,NaN
4,NaN,[],,NaN,Wall Street brokerages expect that Upwork Inc ...,NaN,680aba1ca31cc3e4b50cbe08f461f970,https://www.americanbankingnews.com/wp-content...,NaN,[Internet Content & Information],[],1,2019-12-31T22:30:53+00:00,[Technology],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[UPWK],NaN,Upwork Inc (NASDAQ:UPWK) Expected to Post Quar...,https://newsfilter.io/articles/upwork-inc-nasd...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,NaN,[],,NaN,Cascades Inc (OTCMKTS:CADNF) has received an a...,NaN,22339625b44b360abe8321cc70661362,https://www.americanbankingnews.com/wp-content...,NaN,[],[],0,2019-12-27T21:37:20+00:00,[],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[],NaN,Cascades Inc (OTCMKTS:CADNF) Receives Average ...,https://newsfilter.io/articles/cascades-inc-ot...,NaN,NaN,NaN,NaN,NaN
14695,NaN,[],,NaN,Shares of Tc Pipelines Lp (TSE:TRP) (NYSE:TRP)...,NaN,ddec68bbc6e2e9a6fa7081e21ef31bfa,https://www.americanbankingnews.com/wp-content...,NaN,[Oil & Gas Midstream],[],0,2019-12-27T21:37:20+00:00,[Energy],"{'id': 'TheStockObserver', 'name': 'The Stock ...",[TRP],NaN,Tc Pipelines Lp (TSE:TRP) Given Consensus Reco...,https://newsfilter.io/articles/tc-pipelines-lp...,NaN,NaN,NaN,NaN,NaN
14696,NaN,[],,NaN,Dollar Tree (NASDAQ:DLTR) was notified that TR...,NaN,e86b6507005fda460184a0f36279977e,,NaN,[Discount Stores],[],0,2019-12-27T16:37:00-05:00,[Consumer Defensive],"{'id': 'seekingAlpha', 'name': 'Seeking Alpha'}",[DLTR],NaN,Dollar Tree comments on TRC tender offer,https://newsfilter.io/articles/dollar-tree-com...,NaN,NaN,NaN,NaN,NaN
14697,NaN,[],,NaN,"The Board of Directors (the ""Board"") of The Ch...",NaN,34d162a9b314c52671f3820ca773cde5,,NaN,[],[],0,2019-12-27T16:35:00-05:00,[],"{'id': 'prNewswire', 'name': 'PR Newswire'}",[],NaN,"The China Fund, Inc. Confirms Distributions",https://newsfilter.io/articles/the-china-fund-...,NaN,NaN,NaN,NaN,NaN


In [0]:
input_df['input'] = input_df['title'] + '. ' + input_df['description']

In [0]:
input_df = input_df[['input','output']] 

In [0]:
input_df

,input,output
0,$213.92 Million in Sales Expected for Grand Ca...,1
1,$4.31 Billion in Sales Expected for Aramark (N...,1
2,$0.14 Earnings Per Share Expected for QuinStre...,1
3,Zacks: Analysts Expect Kindred Biosciences Inc...,1
4,Upwork Inc (NASDAQ:UPWK) Expected to Post Quar...,1
...,...,...
14694,Cascades Inc (OTCMKTS:CADNF) Receives Average ...,0
14695,Tc Pipelines Lp (TSE:TRP) Given Consensus Reco...,0
14696,Dollar Tree comments on TRC tender offer. Doll...,0
14697,"The China Fund, Inc. Confirms Distributions. T...",0


In [0]:
input_df.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
input_df['Cleaned_input'] = input_df['input'].apply(normalize, lowercase=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
input_df['Cleaned_input'][0]

'million sale expect grand canyon education inc nasdaq lope quarter analyst forecast grand canyon education inc nasdaq lope announce sale million current quarter accord zacks two analyst make estimate grand canyon education earnings'

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
ids = []
masks=[]
segments=[]

# For every sentence...
for s in tqdm(input_df['Cleaned_input']):
    
    stokens = tokenizer.tokenize(s, lower= True)
    stokens = stokens[0:30]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    ids.append(input_ids)
    masks.append(input_masks)
    segments.append(input_segments)


100%|██████████| 14664/14664 [00:12<00:00, 1130.91it/s]


In [0]:
test_labels = input_df['output'].values

In [0]:
len(test_labels)

14664

In [0]:
ids.shape, segments.shape, masks.shape

((14664, 64), (14664, 64), (14664, 64))

In [0]:
ids = np.array(ids)
masks = np.array(masks)
segments = np.array(segments)

In [0]:
pred = model1.predict([ids, masks, segments])

In [0]:
len(pred)

14664

In [0]:
predictions = []
for i in pred:
  predictions.append(int(tf.math.round(i).numpy()))

In [0]:
count =0 
for i in range(len(predictions)):
  if predictions[i] == test_labels[i]:
    count +=1

In [0]:
len(predictions)

14664

In [0]:
count/len(predictions)

0.9329650845608293

In [0]:
tf.math.confusion_matrix(labels=test_labels , predictions= predictions)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4500,  332],
       [ 651, 9181]], dtype=int32)>